# Gráficos geoespaciales: mapas

Habitualmente necesitamos pintar datos que tienen una referencia geoespacial, por ejemplo:

* Puntos en mapas
* Rutas
* Estados, provincias, países u otras divisiones administrativas de territorios

Para hacerlo, podemos utilizar la librería `folium`, que permite utilizar [`leaflet`](https://leafletjs.com/) dentro de python.

Puedes consultar la documentación de folium [aquí](https://python-visualization.github.io/folium/).

In [1]:
import folium
import pandas as pd

Si te falla el `import folium` es que no tienes la librería instalada. Para hacerlo, corre desde tu terminal:

```
# si usas conda (gestor de paquetes de Anaconda)
conda install -c conda-forge folium

# si usas pip
pip install folium
```

Una vez lo instalas, tendrás que reiniciar el kernel de jupyter antes de volver a probar. Puedes hacerlo desde este notebook en el menú Kernel / Restart.

## Un primer mapa

Para pintar un mapa, llamamos a folium con las opciones que queramos. Por ejemplo, vamos a pintar un mapa de la zona de República Argentina, Madrid

In [2]:
rep_argentina = [40.4438, -3.6857]
folium.Map(location=rep_argentina, zoom_start=15)

Los distintos proveedores de información cartográfica, se denominan `tiles`. Ejemplo: http://maps.stamen.com/toner/#12/40.4299/-3.6692. Podemos personalizarlo con el parámetro `tiles`. Uno que suele quedar bastante bien para pintar datos encima es `cartodbpositron`:

In [3]:
folium.Map(location=rep_argentina, zoom_start=15, tiles='cartodbpositron')

### Ejercicio
Mostrar un mapa usando la fuente `Stamen Toner` con una escala "zoom" de  15

In [15]:
# Respuesta
folium.Map(location=rep_argentina, zoom_start=15, tiles='Stamen Toner')

### No todas las APIs de cartografía son gratuítas
https://www.mapbox.com/

In [4]:
folium.Map(location=rep_argentina, zoom_start=15, tiles='Mapbox')

ValueError: You must pass an API key if using Cloudmade or non-default Mapbox tiles.

## Marcadores

Para añadir una localización, añadimos uno o varios marcadores al mapa. Sobre ellos, podemos personalizar:

* Localización
* Textos en tooltip o popup
* Estilo (forma, color, ...)

Mira más opciones en la [documentación](https://python-visualization.github.io/folium/modules.html#folium.map.Marker).

In [4]:
rep_argentina = [40.446, -3.68]
m = folium.Map(location=rep_argentina, zoom_start=15)
folium.Marker([40.4438, -3.6857], tooltip='EAE Joaquín Costa').add_to(m)
folium.Marker([40.4485,-3.6796], tooltip='EAE Príncipe de Vergada').add_to(m)
m

Vamos a añadir una serie de localizaciones a nuestro mapa. En `dat/` tenemos disponibles las localizaciones de los apartamentos de AirBnB en Madrid, descargados de [Inside AirBnB](http://insideairbnb.com/).

In [5]:
url_AirBNB_Madrid = 'http://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2020-10-17/visualisations/listings.csv'
# listings = pd.read_csv('dat/listings.csv')
listings = pd.read_csv(url_AirBNB_Madrid)
listings.head()

print(listings['room_type'].drop_duplicates())

0          Private room
2       Entire home/apt
266         Shared room
1363         Hotel room
Name: room_type, dtype: object


In [17]:
# Mi mapa base sobre Madrid
map_madrid = folium.Map(location=[40.42, -3.7], zoom_start=13)

# Tomo 100 apartamentos (la muestra completa es muy grande)
listings_sample = listings.sample(100, random_state=1234)

# Los añado a mi mapa
# Voy a usar CircleMarker en lugar de Marker, porque me deja personalizar el color más fácilmente
listings_sample.apply(
    lambda row: folium.CircleMarker([row.latitude, row.longitude], tooltip=row.room_type, radius=3).add_to(map_madrid),
    axis=1
)
map_madrid

#### Ejercicio

Personaliza el mapa anterior para que el color del marcador diferencie por `room_type`.

In [18]:
def room_type_color(room_type):
    if room_type == 'Private room':
        return 'red'
    elif room_type == 'Entire home/apt':
        return 'blue'
    elif room_type == 'Shared room':
        return 'orange'
    elif room_type == 'Hotel room':
        return 'green'

# Mi mapa base sobre Madrid
map_madrid = folium.Map(location=[40.42, -3.7], zoom_start=13)

# Tomo 100 apartamentos (la muestra completa es muy grande)
listings_sample = listings.sample(100, random_state=1234)

# Los añado a mi mapa
# Voy a usar CircleMarker en lugar de Marker, porque me deja personalizar el color más fácilmente
listings_sample.apply(
    lambda row: folium.CircleMarker([row.latitude, row.longitude], tooltip=row.room_type, radius=3, color=room_type_color(row.room_type)).add_to(map_madrid),
    axis=1
)
map_madrid

## Heatmaps

Son útiles para representar densidades.

Vamos a utilizarlo para ver cuál es la concentración de apartamentos por zona.

Si consultamos la [documentación](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap), vemos que necesita un parámetro `data` que debe ser una lista de `[lat, lng]`, pero también acepta un dataframe de pandas con 2 columnas.

In [19]:
from folium.plugins import HeatMap

In [20]:
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')
heatmap = HeatMap(data=listings[["latitude", "longitude"]], radius=15).add_to(m)
m

## Mapa de coropletas

Son mapas sobre los que coloreamos las regiones que contiene en base a la propiedad que queremos explicar.

Vamos a pintar un mapa que represente el precio medio de los alojamientos de cada barrio.

In [21]:
num_per_area = listings.groupby('neighbourhood').id.count().reset_index()
num_per_area = num_per_area.rename(columns={"id": "num"})
num_per_area.head()

,neighbourhood,num
0,Abrantes,55
1,Acacias,191
2,Adelfas,102
3,Aeropuerto,14
4,Aguilas,57


In [22]:
# Datos de origen
url_Neighbourhoods_GeoJSON = 'http://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2020-10-17/visualisations/neighbourhoods.geojson'
# Incializamos el mapa
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')

# Add the color for the chloropleth:
folium.Choropleth(
    geo_data = url_Neighbourhoods_GeoJSON,
    data=num_per_area,
    columns=['neighbourhood', 'num'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlGn'
).add_to(m)

m

#### Ejercicio evaluación

Modifica el mapa anterior para definir los límites de cada color y ver mejor el detalle. P.e. para que de 0 a 10 se vea en una intensidad, de 10 a 100 en otra, de 100 a 1000, ... (es decir, una escala logarítmica)

In [41]:
# Datos de origen
url_Neighbourhoods_GeoJSON = 'http://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2020-10-17/visualisations/neighbourhoods.geojson'
# Incializamos el mapa
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')

bins = list([0, 10, 100, 1000, 10000])

# Add the color for the chloropleth:
folium.Choropleth(
    geo_data = url_Neighbourhoods_GeoJSON,
    data=num_per_area,
    columns=['neighbourhood', 'num'],
    key_on='feature.properties.neighbourhood',
    bins=bins,    
    fill_color='YlGn'    
).add_to(m)

m